# Create a search index in Azure AI Search using the Azure SDK for Python

This notebook steps through creating, loading, and querying an index in Azure AI Search index by calling the azure-search-documents library in the Azure SDK for Python. 

## Install packages and set variables

In [5]:
%pip install azure-search-documents --quiet
%pip install azure-identity --quiet
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from pathlib import Path
from dotenv import load_dotenv
dotenv_path = Path('azure.env')
load_dotenv(dotenv_path=dotenv_path)

# Import Cognitive Search index ENV
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG = os.getenv("AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG")
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")


In [15]:
# Provide variables
search_endpoint: str = AZURE_SEARCH_SERVICE_ENDPOINT
search_api_key: str = AZURE_SEARCH_KEY
index_name: str = AZURE_SEARCH_INDEX
semantic_serach_config: str =AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG




## Create a Complex type index, Semantic Configuration and suggester profile with nested JSON  

In [ ]:
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(search_api_key)

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch
)


# Create a search schema
index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=credential)
fields = [
        SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
        SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

        SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),

        SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
        SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
        SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),
        SimpleField(name="Location", type=SearchFieldDataType.GeographyPoint, facetable=False, filterable=False, sortable=False),

        ComplexField(name="Address", fields=[
            SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
            SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        ]) ,

        ComplexField(name="Rooms", fields=[
            SearchableField(name="Description", type=SearchFieldDataType.String),
            SearchableField(name="Description_fr", type=SearchFieldDataType.String, facetable=False, filterable=False, sortable=False),
            SearchableField(name="Type", type=SearchFieldDataType.String, facetable=False, filterable=False, sortable=False),
            SimpleField(name="BaseRate", type=SearchFieldDataType.Double, facetable=True, filterable=True),
            SearchableField(name="BedOptions", type=SearchFieldDataType.Int64, facetable=False, filterable=False, sortable=False),
            SimpleField(name="SleepsCount", type=SearchFieldDataType.Int64, facetable=False, filterable=False,),
            SimpleField(name="SmokingAllowed", type=SearchFieldDataType.Boolean, facetable=False, filterable=False),
            SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=False, filterable=False, sortable=False),




        ],
        collection=True,
        )
    ]

scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['Tags', 'Address/City', 'Address/Country']}]


semantic_config = SemanticConfiguration(
    name=AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG,
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Description")],
        content_fields=[SemanticField(field_name="Description")]
    )
)
# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

semantic_settings = SemanticSearch(configurations=[semantic_config])
# Create the search index=
index = SearchIndex(name=index_name, fields=fields, suggesters=suggester, scoring_profiles=scoring_profiles,semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

## Read the JSON file

In [ ]:
import json
with open("HotelsData_toAzureSearch.JSON") as user_file:
    file_contents = user_file.read()
    parsed_json = json.loads(file_contents)

## Upload the documents from the JSON file

In [ ]:
search_client = SearchClient(endpoint=search_endpoint,
                      index_name=index_name,
                      credential=credential)
try:
    result = search_client.upload_documents(documents=parsed_json)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

    index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=credential)

**Note:** 

**1 .** After the "Upload of new document succeeded: True" message, you can execute the "Autogen-multi-agentchat.ipynb" notebook to see the agent execution on our created index.

**2 .** Optionally, you can run the below code blocks to run the search queries on the created index

## Run a query

In [ ]:
# Run a term query
results =  search_client.search(query_type='simple',
    search_text="wifi" ,
    select='HotelName,Description,Tags',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

## Add a filter

In [ ]:
# Add a filter
results = search_client.search(
    search_text="hotels", 
    select='HotelId,HotelName,Rating', 
    filter='Rating gt 4', 
    order_by='Rating desc')

for result in results:
    print("{}: {} - {} rating".format(result["HotelId"], result["HotelName"], result["Rating"]))

## Scope a query to specific searchable fields

In [ ]:
results = search_client.search(
    search_text="sublime", 
    search_fields=['HotelName'], 
    select='HotelId,HotelName')

for result in results:
    print("{}: {}".format(result["HotelId"], result["HotelName"]))

## Return facets

In [ ]:
# Return facets
results = search_client.search(search_text="*", facets=["Category"])

facets = results.get_facets()

for facet in facets["Category"]:
    print("    {}".format(facet))

## Look up a document 

In [ ]:
# Look up a specific document by ID
result = search_client.get_document(key="31")

print("Details for hotel '31' are:")
print("Name: {}".format(result["HotelName"]))
print("Rating: {}".format(result["Rating"]))
print("Category: {}".format(result["Category"]))

## Autocomplete a query

In [ ]:
# Autocomplete a query
search_suggestion = 'sa'
results = search_client.autocomplete(
    search_text=search_suggestion, 
    suggester_name="sg",
    mode='twoTerms')

print("Autocomplete for:", search_suggestion)
for result in results:
    print (result['text'])